In [ ]:
import json
import csv
import os

json_file_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard\pacz.json"
csv_folder_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
csv_file_name = "paczkomaty.csv"
csv_file_path = os.path.join(csv_folder_path, csv_file_name)

with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

parcel_lockers = []
all_tag_keys = set()

for element in data.get('elements', []):
    if element.get('tags', {}).get('amenity') == 'parcel_locker':
        locker_data = {
            'id': element.get('id'),
            'type': element.get('type')
        }
        if element.get('type') == 'node':
            locker_data['latitude'] = element.get('lat')
            locker_data['longitude'] = element.get('lon')
        else:
            locker_data['latitude'] = element.get('lat')
            locker_data['longitude'] = element.get('lon')
            if 'center' in element:
                 locker_data['latitude'] = element['center'].get('lat')
                 locker_data['longitude'] = element['center'].get('lon')

        if 'tags' in element:
            for key, value in element['tags'].items():
                locker_data[key] = value
                all_tag_keys.add(key)
        
        parcel_lockers.append(locker_data)

if parcel_lockers:
    base_headers = ['id', 'type', 'latitude', 'longitude']
    sorted_tag_keys = sorted(list(all_tag_keys))
    
    final_headers = base_headers[:]
    for key in sorted_tag_keys:
        if key not in final_headers:
            final_headers.append(key)

    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=final_headers, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(parcel_lockers)

Wczytywanie danych z pliku: C:\Users\anton\OneDrive\Pulpit\Mastercard\pacz.json
Przetwarzanie elementów...
Znaleziono 19646 paczkomatów.
Zapisywanie danych do pliku CSV: C:\Users\anton\OneDrive\Pulpit\Mastercard\paczkomaty.csv
Dane zostały pomyślnie zapisane do C:\Users\anton\OneDrive\Pulpit\Mastercard\paczkomaty.csv


In [ ]:
import csv
import os

input_csv_file_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard\paczkomaty.csv"
output_csv_folder_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
output_csv_file_name = "final.csv"
output_csv_file_path = os.path.join(output_csv_folder_path, output_csv_file_name)

final_columns = ['id', 'latitude', 'longitude', 'brand']
final_data = []

with open(input_csv_file_path, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    if reader.fieldnames: # Sprawdzenie czy nagłówki istnieją
        for row in reader:
            processed_row = {}
            for col in final_columns:
                processed_row[col] = row.get(col, '') 
            final_data.append(processed_row)

if final_data:
    with open(output_csv_file_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=final_columns)
        writer.writeheader()
        writer.writerows(final_data)

Wczytywanie danych z pliku: C:\Users\anton\OneDrive\Pulpit\Mastercard\paczkomaty.csv
Przetworzono 19646 wierszy.
Zapisywanie danych do finalnego pliku CSV: C:\Users\anton\OneDrive\Pulpit\Mastercard\final.csv
Dane zostały pomyślnie zapisane do C:\Users\anton\OneDrive\Pulpit\Mastercard\final.csv


In [ ]:
import csv
import os

input_csv_file_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard\final.csv"
output_csv_folder_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
output_csv_file_name = "final1.csv"
output_csv_file_path = os.path.join(output_csv_folder_path, output_csv_file_name)

output_columns = ['id', 'latitude', 'longitude', 'brand']
transformed_data = []

with open(input_csv_file_path, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    if reader.fieldnames: # Sprawdzenie czy nagłówki istnieją
        for row in reader:
            original_brand = row.get('brand', '').strip()
            modified_brand = original_brand

            if "InPost" in original_brand:
                modified_brand = "Paczkomat InPost"
            elif "DHL" in original_brand:
                modified_brand = "DHL BOX 24/7"
            elif "DPD" in original_brand:
                modified_brand = "DPD Pickup"
            elif original_brand == "Allegro One Box":
                modified_brand = "Allegro One Box"
            elif original_brand == "Orlen Paczka":
                modified_brand = "Orlen Paczka"

            transformed_row = {
                'id': row.get('id'),
                'latitude': row.get('latitude'),
                'longitude': row.get('longitude'),
                'brand': modified_brand
            }
            transformed_data.append(transformed_row)

if transformed_data:
    with open(output_csv_file_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=output_columns)
        writer.writeheader()
        writer.writerows(transformed_data)

Wczytywanie danych z pliku: C:\Users\anton\OneDrive\Pulpit\Mastercard\final.csv
Przetworzono 19646 wierszy.
Zapisywanie zmodyfikowanych danych do pliku CSV: C:\Users\anton\OneDrive\Pulpit\Mastercard\final1.csv
Dane zostały pomyślnie zapisane do C:\Users\anton\OneDrive\Pulpit\Mastercard\final1.csv


In [ ]:
import csv
import os

input_csv_file_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard\final1.csv"
output_csv_folder_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
output_csv_file_name = "final2.csv"
output_csv_file_path = os.path.join(output_csv_folder_path, output_csv_file_name)

output_columns = ['id', 'latitude', 'longitude', 'brand']
brands_to_remove = ["Paczkomat", "SwipBox"]
filtered_data = []

with open(input_csv_file_path, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    if reader.fieldnames:
        for row in reader:
            current_brand = row.get('brand', '').strip()
            if current_brand not in brands_to_remove:
                filtered_data.append(row)

if filtered_data:
    with open(output_csv_file_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=output_columns)
        writer.writeheader()
        writer.writerows(filtered_data)

Wczytywanie danych z pliku: C:\Users\anton\OneDrive\Pulpit\Mastercard\final1.csv
Usunięto 4 wierszy.
Pozostało 19642 wierszy do zapisania.
Zapisywanie przefiltrowanych danych do pliku CSV: C:\Users\anton\OneDrive\Pulpit\Mastercard\final2.csv
Dane zostały pomyślnie zapisane do C:\Users\anton\OneDrive\Pulpit\Mastercard\final2.csv


In [ ]:
import pandas as pd
import geopandas
from shapely.geometry import Point, Polygon
import os
import time
import math

base_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
grid_file_path = os.path.join(base_path, "better_coords.csv")
lockers_file_path = os.path.join(base_path, "final2.csv")
output_file_path = os.path.join(base_path, "grid_2km_with_locker_counts.csv")

HALF_SIDE_KM = 1.0
KM_PER_DEGREE_LAT = 111.132
KM_PER_DEGREE_LON_BASE = 111.320

def calculate_delta_lat_degrees(km):
    return km / KM_PER_DEGREE_LAT

def calculate_delta_lon_degrees(km, center_lat_degrees):
    center_lat_radians = math.radians(center_lat_degrees)
    cos_lat = math.cos(center_lat_radians)
    if abs(cos_lat) < 1e-9:
        return float('inf')
    km_per_degree_lon_at_lat = KM_PER_DEGREE_LON_BASE * cos_lat
    if km_per_degree_lon_at_lat == 0:
        return float('inf')
    return km / km_per_degree_lon_at_lat

print("Krok 1: Wczytywanie paczkomatów...")
lockers_df = pd.read_csv(lockers_file_path, usecols=['id', 'latitude', 'longitude', 'brand'], dtype={'id': str})
lockers_df.dropna(subset=['latitude', 'longitude', 'brand'], inplace=True)
lockers_df['brand'] = lockers_df['brand'].astype(str).str.strip()
lockers_df = lockers_df[lockers_df['brand'] != '']
lockers_df['latitude'] = pd.to_numeric(lockers_df['latitude'], errors='coerce')
lockers_df['longitude'] = pd.to_numeric(lockers_df['longitude'], errors='coerce')
lockers_df.dropna(subset=['latitude', 'longitude'], inplace=True)
geometry_lockers = [Point(xy) for xy in zip(lockers_df['longitude'], lockers_df['latitude'])]
lockers_gdf = geopandas.GeoDataFrame(lockers_df, geometry=geometry_lockers, crs="EPSG:4326")
print(f"Wczytano {len(lockers_gdf)} paczkomatów.")

if lockers_gdf.empty:
    print("Brak danych o paczkomatach. Przerywam.")
    exit()

print("Krok 2: Wczytywanie siatki i tworzenie poligonów...")
grid_df_cols_to_use = ['longitude', 'latitude', 'geometry']
grid_df = pd.read_csv(grid_file_path, usecols=grid_df_cols_to_use)
grid_df.dropna(subset=['latitude', 'longitude'], inplace=True)
grid_df['latitude'] = pd.to_numeric(grid_df['latitude'], errors='coerce')
grid_df['longitude'] = pd.to_numeric(grid_df['longitude'], errors='coerce')
grid_df.dropna(subset=['latitude', 'longitude'], inplace=True)
grid_df.reset_index(drop=True, inplace=True)
grid_df['grid_id'] = grid_df.index

if grid_df.empty:
    print("Brak danych siatki. Przerywam.")
    exit()

print(f"Wczytano {len(grid_df)} środków kwadratów. Tworzenie poligonów...")
grid_polygons = []
total_grids = len(grid_df)
for index, row in grid_df.iterrows():
    center_lon = row['longitude']
    center_lat = row['latitude']
    delta_lat_deg = calculate_delta_lat_degrees(HALF_SIDE_KM)
    delta_lon_deg = calculate_delta_lon_degrees(HALF_SIDE_KM, center_lat)
    min_lon, max_lon = center_lon - delta_lon_deg, center_lon + delta_lon_deg
    min_lat, max_lat = center_lat - delta_lat_deg, center_lat + delta_lat_deg
    polygon = Polygon([(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat), (min_lon, min_lat)])
    grid_polygons.append(polygon)
    if (index + 1) % 10000 == 0 or (index + 1) == total_grids:
        print(f"Utworzono geometrię dla {index + 1}/{total_grids} kwadratów...")

grid_gdf = geopandas.GeoDataFrame(grid_df, geometry=grid_polygons, crs="EPSG:4326")
print(f"Utworzono {len(grid_gdf)} poligonów.")

print("Krok 3: Łączenie przestrzenne...")
start_sjoin_time = time.time()
joined_gdf = geopandas.sjoin(lockers_gdf, grid_gdf, how="inner", predicate="within", lsuffix="locker", rsuffix="grid")
print(f"Łączenie przestrzenne zakończone w {time.time() - start_sjoin_time:.2f}s. Znaleziono {len(joined_gdf)} paczkomatów w kwadratach.")

if joined_gdf.empty:
    result_df = grid_df.copy()
    unique_brands_from_lockers = [str(b).strip() for b in lockers_gdf['brand'].unique() if str(b).strip()]
    for brand_name in unique_brands_from_lockers:
        result_df[brand_name] = 0
else:
    print("Krok 4: Agregacja...")
    group_column = 'grid_id'
    count_column = 'id'
    counts_df = joined_gdf.groupby([group_column, 'brand'])[count_column].count().unstack(fill_value=0)
    result_df = grid_df.merge(counts_df, on='grid_id', how='left')
    brand_columns_to_fill = [str(b).strip() for b in lockers_gdf['brand'].unique() if str(b).strip()]
    for col in brand_columns_to_fill:
        if col not in result_df.columns:
            result_df[col] = 0
        else:
            result_df[col] = result_df[col].fillna(0).astype(int)
    print("Agregacja zakończona.")

print("Krok 5: Zapisywanie wyników...")
cols_to_save = list(result_df.columns)
if 'geometry' in cols_to_save and not all(isinstance(g, str) for g in result_df['geometry'].dropna()):
    cols_to_save.remove('geometry')
result_df[cols_to_save].to_csv(output_file_path, index=False, encoding='utf-8')
print(f"Wyniki zapisano do {output_file_path}")

Krok 1: Wczytywanie danych o paczkomatach z C:\Users\anton\OneDrive\Pulpit\Mastercard\final2.csv...
Wczytano 19446 paczkomatów jako GeoDataFrame.
Czas wczytywania paczkomatów: 3.98s.

Krok 2: Wczytywanie siatki z C:\Users\anton\OneDrive\Pulpit\Mastercard\better_coords.csv i tworzenie poligonów (2km x 2km)...
Wczytano 77273 środków kwadratów z pliku siatki.
Tworzenie geometrii poligonów dla kwadratów (to może chwilę potrwać)...
Utworzono geometrię dla 5000/77273 kwadratów...
Utworzono geometrię dla 10000/77273 kwadratów...
Utworzono geometrię dla 15000/77273 kwadratów...
Utworzono geometrię dla 20000/77273 kwadratów...
Utworzono geometrię dla 25000/77273 kwadratów...
Utworzono geometrię dla 30000/77273 kwadratów...
Utworzono geometrię dla 35000/77273 kwadratów...
Utworzono geometrię dla 40000/77273 kwadratów...
Utworzono geometrię dla 45000/77273 kwadratów...
Utworzono geometrię dla 50000/77273 kwadratów...
Utworzono geometrię dla 55000/77273 kwadratów...
Utworzono geometrię dla 60000/7